## Is it a bear?
[Reference Link](https://github.com/fastai/fastbook/blob/master/02_production.ipynb)

In [ ]:
#NB: Kaggle requires phone verification to use the internet or a GPU. If you haven't done that yet, the cell below will fail
#    This code is only here to check that your internet is enabled. It doesn't do anything else.
#    Here's a help thread on getting your phone number verified: https://www.kaggle.com/product-feedback/135367

import socket,warnings
try:
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
except socket.error as ex: raise Exception("STOP: No internet. Click '>|' in top right and set 'Internet' switch to on")

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

## Step 1: Download images of bear and non-bear

In [ ]:
# Skip this cell if you already have duckduckgo_search installed
!pip install -Uqq ddgs

In [ ]:
from ddgs import DDGS
from fastcore.all import *

def search_images(keywords, max_images=200): return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')

Let's start by searching for a bear photo and seeing what kind of result we get. We'll start by getting URLs from a search:

In [ ]:
urls = search_images('bear photos', max_images=1)
urls[0]

...and then download a URL and take a look at it:

In [ ]:
from fastdownload import download_url
dest = 'bear.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Now let's do the same with "grizzly bear photos":

In [ ]:
download_url(search_images('grizzly bear photos', max_images=1)[0], 'grizzly_bear.jpg', show_progress=False)
Image.open('grizzly_bear.jpg').to_thumb(256,256)

Our searches seem to be giving reasonable results, so let's grab 200 examples of each of 'grizzly','black','teddy','polar' photos, and save each group of photos to a different folder:

In [ ]:
searches = 'grizzly','black','teddy','polar'
path = Path('bear')
from time import sleep

if not path.exists():
    path.mkdir()
    for o in searches:
        dest = (path/o)
        dest.mkdir(exist_ok=True, parents=True)
        download_images(dest, urls=search_images(f'{o} bear photo'))

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))


In [ ]:
dls = bears.dataloaders(path)

In [ ]:
#View datablocks

dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#check resize options

bears = bears.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
#changing the pad mode
bears = bears.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = bears.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)


To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
#changing the scal size
# replace Resize with RandomResizedCrop, which is the transform that provides the behavior we just described. The most important parameter to pass in is min_scale, which determines how much of the image to select at minimum each time:
bears = bears.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
#Data Augmentation
bears = bears.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = bears.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)


## Step 2: Train our model

In [ ]:
## Step 2: Train our model
##We don't have a lot of data for our problem (150 pictures of each sort of bear at most), so to train our model,
## we'll use RandomResizedCrop with an image size of 224 px, which is fairly standard for image classification, and default aug_transforms
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
#Interpret the confusion matrix
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#validate the top losses
interp.plot_top_losses(5, nrows=1)

In [ ]:
#Lets clean up
from fastai.vision.widgets import ImageClassifierCleaner
cleaner = ImageClassifierCleaner(learn)
cleaner


In [ ]:
#clear the false data
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

## Step 3: Use our model (and build your own!)

Let's see what our model thinks about that bear we downloaded at the start:

In [ ]:
title,_,probs = learn.predict(PILImage.create('bear.jpg'))
print(f"This is a: {title}.")
print(f"Probability: {probs[0]:.4f}")

# Step 4: Export our model

In [ ]:
learn.export('model.pkl')

In [ ]:
#Validate if downloaded or not
path = Path()
path.ls(file_exts='.pkl')